<a href="https://colab.research.google.com/github/jazoza/cultural-data-analysis/blob/main/HDA-chapter-1/HDA_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Humanities Data Analysis: Case studies with Python
--------------------------------------------------
book by Folgert Karsdorp, Mike Kestemont & Allen Riddell
## Code from Chapter 1: Introduction
From https://www.humanitiesdataanalysis.org/introduction-cook-books/notebook.html#an-exploratory-data-analysis-of-the-united-states-culinary-history

In [ ]:
x = 100

In [ ]:
saying = "It's turtles all the way down"

In [ ]:
print(saying[0])

In [ ]:
string = "Python"
for character in string:
    print(character)

In [ ]:
numbers = [1, 1, 2, 3, 5, 8]
words = ["This", "is", "a", "list", "of", "strings"]

In [ ]:
print(numbers[0])
print(numbers[-1])  # use -1 to retrieve the last item in a sequence
print(words[3:])  # use slice syntax to retrieve a subsequence

In [ ]:
packages = {'matplotlib': 'Matplotlib is a Python 2D plotting library',
            'pandas': 'Pandas is a Python library for data analysis',
            'scikit-learn': 'Scikit-learn helps with Machine Learning in Python'}

In [ ]:
print(packages['pandas'])

In [ ]:
packages = {"matplotlib", "pandas", "scikit-learn"}

In [ ]:
words = ["move", "slowly", "and", "fix", "things"]
for word in words:
    if "i" in word:
        print(word)

In [ ]:
import math

In [ ]:
print(math.log(2.7183))
print(math.sqrt(2))

In [ ]:
def count_ing(strings):
    count = 0
    for string in strings:
        if string.endswith("ing"):
            count += 1
    return count

words = [
    "coding", "is", "about", "developing", "logical", "event", "sequences"
]
print(count_ing(words))

In [ ]:
!wget https://raw.githubusercontent.com/jazoza/cultural-data-analysis/refs/heads/main/HDA-chapter-1/data/aesop-wolf-dog.txt

In [ ]:
f = open("aesop-wolf-dog.txt")  # open a file
text = f.read()  # read the contents of a file
f.close()  # close the connection to the file
print(text)  # print the contents of the file

In [ ]:
import pandas as pd

In [ ]:
!wget https://raw.githubusercontent.com/jazoza/cultural-data-analysis/refs/heads/main/HDA-chapter-1/data/feeding-america.csv

In [ ]:
df = pd.read_csv("feeding-america.csv", index_col='date')

In [ ]:
df.head()

In [ ]:
print(len(df))

In [ ]:
print(df['recipe_class'].unique())

In [ ]:
df['recipe_class'].value_counts()

In [ ]:
df['recipe_class'].value_counts().plot(kind='bar', color="C0", width=0.1);

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
grouped = df.groupby('date')  # group all rows from the same year
recipe_counts = grouped.size()  # compute the size of each group
recipe_counts.plot(style='o', xlim=(1810, 1930))  # plot the group size
plt.ylabel("number of recipes")  # add a label to the y-axis
plt.xlabel("year of publication") # add a label to the x-axis

In [ ]:
# split ingredient strings into lists
ingredients = df['ingredients'].str.split(';')
# group all rows from the same year
groups = ingredients.groupby('date')
# merge the lists from the same year
ingredients = groups.sum()
# compute counts per year
ingredients = ingredients.apply(lambda x: pd.Series(x).value_counts())
# normalise the counts
ingredients = ingredients.divide(recipe_counts, 0)

In [ ]:
ingredients.head()

In [ ]:
ax = ingredients['tomato'].plot(style='o', xlim=(1810, 1930))
ax.set_ylabel("fraction of recipes")
ax.set_xlabel("year of publication");

In [ ]:
import scipy.stats

In [ ]:
def plot_trend(column, df, line_color='grey', xlim=(1810, 1930)):
    slope, intercept, _, _, _ = scipy.stats.linregress(
        df.index, df[column].fillna(0).values)
    ax = df[column].plot(style='o', label=column)
    ax.plot(df.index, intercept + slope * df.index, '--',
             color=line_color, label='_nolegend_')
    ax.set_ylabel("fraction of recipes")
    ax.set_xlabel("year of publication")
    ax.set_xlim(xlim)

In [ ]:
plot_trend('tomato', ingredients)

In [ ]:
plot_trend('baking powder', ingredients)
plot_trend('yeast', ingredients)
plt.legend();  # add a legend to the plot

In [ ]:
plot_trend('nutmeg', ingredients)

In [ ]:
from sklearn.feature_selection import chi2

In [ ]:
# Transform the index into a list of labels, in which each label
# indicates whether a row stems from before or after the Civil War:
labels = ['Pre-Civil War' if year < 1864 else 'Post-Civil War' for year in ingredients.index]
# replace missing values with zero (.fillna(0)),
# and compute the chi2 statistic:
keyness, _ = chi2(ingredients.fillna(0), labels)
# Turn keyness values into a Series, and sort in descending order:
keyness = pd.Series(keyness, index=ingredients.columns).sort_values(ascending=False)

In [ ]:
keyness.head(n=10)

In [ ]:
# step 1: compute summed ingredient counts per year
counts = df['ingredients'].str.split(';').groupby(
    'date').sum().apply(pd.Series.value_counts).fillna(0)

In [ ]:
# step 2: construct frequency rankings for pre- and post-war years
pre_cw = counts[counts.index < 1864].sum().rank(method='dense', pct=True)
post_cw = counts[counts.index > 1864].sum().rank(method='dense', pct=True)

In [ ]:
# step 3: merge the pre- and post-war data frames
rankings = pd.DataFrame({'Pre-Civil War': pre_cw, 'Post-Civil War': post_cw})

In [ ]:
# step 4: produce the plot
fig = plt.figure(figsize=(10, 6))
plt.scatter(rankings['Post-Civil War'], rankings['Pre-Civil War'],
            c=rankings['Pre-Civil War'] - rankings['Post-Civil War'],
            alpha=0.7)

# Add annotations of the 20 most distinctive ingredients
for i, row in rankings.loc[keyness.head(20).index].iterrows():
    plt.annotate(i, xy=(row['Post-Civil War'], row['Pre-Civil War']))

plt.xlabel("Frequency rank Post-Civil War")
plt.ylabel("Frequency rank Pre-Civil War");

In [ ]:
df['ethnicgroup'].value_counts(dropna=False).head(10)

In [ ]:
grouped = df.groupby(level='date')
# compute the number of unique ethnic groups per year,
# divided by the number of books
n_groups = grouped['ethnicgroup'].nunique() / grouped['book_id'].nunique()
n_groups.plot(style='o')

# add a least square line as reference
slope, intercept, _, _, _ = scipy.stats.linregress(
    n_groups.index, n_groups.fillna(0).values)

# create the plot
plt.plot(
    n_groups.index, intercept + slope * n_groups.index, '--', color="grey")
plt.xlim(1810, 1930)
plt.ylabel("Average number of ethnic groups")
plt.xlabel("Year of publication");

In [ ]:
# step 1: add a new column indicating for each recipe whether
#         we have information about its ethnic group
df['foreign'] = df['ethnicgroup'].notnull()

# step 2: construct frequency rankings for foreign and general recipes
counts = df.groupby('foreign')['ingredients'].apply(
    ';'.join).str.split(';').apply(pd.Series.value_counts).fillna(0)

foreign_counts = counts.iloc[1].rank(method='dense', pct=True)
general_counts = counts.iloc[0].rank(method='dense', pct=True)

In [ ]:
# step 3: merge the foreign and general data frames
rankings = pd.DataFrame({'foreign': foreign_counts, 'general': general_counts})

In [ ]:
# step 4: compute the keyness of ingredients in foreign recipes
#         as the difference in frequency ranks
keyness = (rankings['foreign'] - rankings['general']).sort_values(ascending=False)

In [ ]:
# step 5: produce the plot
fig = plt.figure(figsize=(10, 6))
plt.scatter(rankings['general'], rankings['foreign'],
            c=rankings['foreign'] - rankings['general'],
            alpha=0.7)

for i, row in rankings.loc[keyness.head(10).index].iterrows():
    plt.annotate(i, xy=(row['general'], row['foreign']))

plt.xlabel("Frequency rank general recipes")
plt.ylabel("Frequency rank foreign recipes");